In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\ahmed\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')


In [2]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [5]:
from matplotlib import pyplot as plt

random_file = 'image00053'

faceModule = mediapipe.solutions.face_mesh
# loading image and its correspinding mat file
with faceModule.FaceMesh(static_image_mode=True) as faces:
    # loading the image
    image = cv2.imread('C:/Users/ahmed/Desktop/ITI/ML Supervised/AFLW2000/'+random_file+'.jpg')
    # processing the face to extract the landmark points (468 point) for each x,y,z
    results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks != None: 
      # looping over the faces in the image
      for face in results.multi_face_landmarks:
          for landmark in face.landmark:
              x = landmark.x
              y = landmark.y
              # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
              shape = image.shape 
              relative_x = int(x * shape[1])
              relative_y = int(y * shape[0])
              # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
              cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
      cv2.imshow('image',image)

: 

In [5]:
random_file = 'image00053'

image = cv2.imread('C:/Users/ahmed/Desktop/ITI/ML Supervised/AFLW2000/'+random_file+'.jpg')
# loading the mat file to extract the 3 angels of the face (pitch,yaw,roll)
mat_file = sio.loadmat('C:/Users/ahmed/Desktop/ITI/ML Supervised/AFLW2000/'+random_file+'.mat')
# extracting the labels 3 angels
pose_para = mat_file["Pose_Para"][0][:3]
pitch = pose_para[0]
yaw = pose_para[1]
roll = pose_para[2]
cv2.imshow('nn',draw_axis(image,pitch,yaw,roll))
plt.show()

: 

In [3]:

#spliting data for the training into image and poses

dataset_dir = 'C:/Users/ahmed/Desktop/ITI/ML Supervised/AFLW2000/'

# Get a list of image files in the dataset directory
image_files = [file for file in os.listdir(dataset_dir) if file.endswith('.jpg')]

# Initialize arrays to store images and corresponding poses
images = []
poses = []
i= 0
# Load images and poses from the dataset
for image_file in image_files:
    # Load image
    image_path = os.path.join(dataset_dir, image_file)
    image = cv2.imread(image_path)
    X = []

    faceModule = mediapipe.solutions.face_mesh
    # loading image and its correspinding mat file
    with faceModule.FaceMesh(static_image_mode=True) as faces:
      results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      if results.multi_face_landmarks != None:
        # looping over the faces in the image
        for face in results.multi_face_landmarks:
            for landmark in face.landmark:
                x = landmark.x
                y = landmark.y
                # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
                shape = image.shape
                relative_x = int(x * shape[1])
                relative_y = int(y * shape[0])
                X.append(relative_x)
                X.append(relative_y)
    if i ==0 :
      images.append(X)
      images = np.array(images)
      i+=1
    else:  
      X = np.array(X)
      if X.shape == (0,):
        continue
      images = np.concatenate((images,X.reshape(1,936)),axis=0)
      # print(images)
    # Load corresponding pose parameters from .mat file
    mat_file_path = os.path.join(dataset_dir, image_file.replace('.jpg', '.mat'))
    mat_file = sio.loadmat(mat_file_path)
    pose_parameters = mat_file["Pose_Para"][0][:3]  # Extract pitch, yaw, and roll
    poses.append(pose_parameters)

poses = np.array(poses)
print(images.shape)
# Print the shape of the arrays
print("Poses shape:", poses.shape)

(1853, 936)
Poses shape: (1853, 3)


In [4]:
from sklearn.model_selection import train_test_split

# Split the data into training, and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    images, poses, test_size=0.2, random_state=42
)


print("Train images shape:", X_train.shape)
print("Test images shape:", X_test.shape)
print("Train poses shape:", y_train.shape)
print("Test poses shape:", y_test.shape)

Train images shape: (1482, 936)
Test images shape: (371, 936)
Train poses shape: (1482, 3)
Test poses shape: (371, 3)


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)


r2 = r2_score(y_test, predictions)
print(f'R2 Score: {r2}')

R2 Score: -3.2505294764466437


In [8]:
from sklearn.linear_model import Ridge

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
grid_search = GridSearchCV(Ridge(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_alpha = grid_search.best_params_['alpha']
print(f"Best alpha: {best_alpha}")

model = Ridge(alpha=best_alpha)
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)

r2 = r2_score(y_test, predictions)
print(f'R2 Score: {r2}')


Best alpha: 5.0
R2 Score: 0.15975271516759984


In [11]:
from sklearn.linear_model import Lasso

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
grid_search = GridSearchCV(Lasso(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_alpha = grid_search.best_params_['alpha']
print(f"Best alpha: {best_alpha}")

model = Lasso(alpha=best_alpha)
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)

r2 = r2_score(y_test, predictions)
print(f'R2 Score: {r2}')


Best alpha: 0.1
R2 Score: 0.636381120345517


In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_scaled)

r2 = r2_score(y_test, predictions)
print(f'R2 Score: {r2}')

R2 Score: -4.67888983073012


In [ ]:
from sklearn.ensemble import RandomForestRegressor

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

r2 = r2_score(y_test, predictions)
print(f'Improved R2 Score: {r2}')

In [7]:
from sklearn.neighbors import KNeighborsRegressor


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_k = grid_search.best_params_['n_neighbors']
print(f"Best number of neighbors: {best_k}")

model = KNeighborsRegressor(n_neighbors=best_k)
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)

r2 = r2_score(y_test, predictions)
print(f'R2 Score: {r2}')

Best number of neighbors: 11
R2 Score: 0.7755195418884603


In [ ]:
from sklearn.svm import SVR

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svr = SVR()

param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 0.5, 1],
}


grid_search = GridSearchCV(svr, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_kernel = grid_search.best_params_['kernel']
best_C = grid_search.best_params_['C']
best_epsilon = grid_search.best_params_['epsilon']

print(f"Best kernel: {best_kernel}")
print(f"Best C: {best_C}")
print(f"Best epsilon: {best_epsilon}")

svr_best = SVR(kernel=best_kernel, C=best_C, epsilon=best_epsilon)
svr_best.fit(X_train_scaled, y_train)

predictions = svr_best.predict(X_test_scaled)

r2 = r2_score(y_test, predictions)
print(f'R2 Score: {r2}')


In [18]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {
    'estimator__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'estimator__C': [0.1, 1, 10],
    'estimator__epsilon': [0.01, 0.1, 0.5, 1],
}

grid_search = GridSearchCV(MultiOutputRegressor(SVR()), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_kernel = grid_search.best_params_['estimator__kernel']
best_C = grid_search.best_params_['estimator__C']
best_epsilon = grid_search.best_params_['estimator__epsilon']

print(f"Best kernel: {best_kernel}")
print(f"Best C: {best_C}")
print(f"Best epsilon: {best_epsilon}")

svr_multioutput_tuned = MultiOutputRegressor(SVR(kernel=best_kernel, C=best_C, epsilon=best_epsilon))
svr_multioutput_tuned.fit(X_train_scaled, y_train)
predictions_multioutput_tuned = svr_multioutput_tuned.predict(X_test_scaled)

r2_multioutput_tuned = r2_score(y_test, predictions_multioutput_tuned)
print(f'R2 Score (Multi-Output Tuned): {r2_multioutput_tuned}')

In [8]:
def preprocess_image(image):
    # Preprocess the image as needed (resize, normalization, etc.)
    # Replace this with your actual preprocessing steps
    processed_image =  cv2.resize(image, (450, 450))
    return processed_image

In [9]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [1]:
from statistics import mean 
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Preprocess the frame
    processed_frame = preprocess_image(frame)

    # Reshape the frame to match the input shape expected by the model
    input_data = np.expand_dims(processed_frame, axis=0)

    faceModule = mediapipe.solutions.face_mesh
    # loading image and its correspinding mat file
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        # loading the image
        image = processed_frame
        # processing the face to extract the landmark points (468 point) for each x,y,z
        x_list = []
        y_list = []
        all = []
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
        # looping over the faces in the image
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    
                    # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    x_list.append(relative_x)

                    relative_y = int(y * shape[0])
                    y_list.append(relative_y)
                    all.append(relative_x)
                    all.append(relative_y)
                    # cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)

                    # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
    dx = mean(x_list)    
    dy = mean(y_list)       


    all = np.array(all)
    if all == (0,):
        continue
    else:
    # Perform prediction using the model
        predictions = model.predict(all.reshape(1,936))

    # Extract the predicted pitch, yaw, and roll
        predicted_pitch, predicted_yaw, predicted_roll = predictions[0]

    # Draw axes lines on the frame
        frame_with_axes = draw_axis(image.copy(), predicted_pitch, predicted_yaw, predicted_roll,dx,dy)


    # Display the frame with axes
        cv2.imshow('Pose Prediction with Axes', frame_with_axes)

    # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined